In [118]:
import numpy as np


class Aligner:
    
    
    def __init__(self, seq1, seq2, gapPenalty, missPenalty, matchScore):
        self.seq1 = seq1
        self.seq2 = seq2
        self.gapPenalty = gapPenalty
        self.missPenalty = missPenalty
        self.matchScore = matchScore
        self.alignMatrix = np.zeros((len(self.seq1)+1, len(self.seq2)+1), dtype=int)
        self.traceBackMatrix = np.zeros((len(self.seq1)+1, len(self.seq2)+1), dtype='U4')
        self.indexToTrace = {
            0: "d",
            1: "l",
            2: "u"
        }
        self.finalScore = 0
        self.identity = 0
        
    
    
    def getValue(self, i, j):
        if self.seq1[i-1] == self.seq2[j-1]:
            missOrMatch = self.matchScore
        else:
            missOrMatch = self.missPenalty
        
        possibleValues = [
            self.alignMatrix[i-1][j-1] + missOrMatch,
            self.alignMatrix[i][j-1] + self.gapPenalty,
            self.alignMatrix[i-1][j] + self.gapPenalty
        ]
        
        return max(possibleValues), possibleValues.index(max(possibleValues))
    
    
    
    def align(self):
        
        for row in self.traceBackMatrix:
            row[0] = "u"
        
        for i in range(len(self.traceBackMatrix[0])):
            self.traceBackMatrix[0][i] = "l"
        self.traceBackMatrix[0][0] = "f"
        
        accGap = 0
        for row in self.alignMatrix:
            row[0] = accGap
            accGap += self.gapPenalty
        
        accGap = 0
        for i in range(len(self.alignMatrix[0])):
            self.alignMatrix[0][i] = accGap
            accGap += self.gapPenalty
        

        for i,j in np.ndindex(self.alignMatrix.shape):
            if i == 0:
                continue
            if j == 0:
                continue

            self.alignMatrix[i][j], index = self.getValue(i,j)
            self.traceBackMatrix[i][j] = self.indexToTrace[index]
        
        self.finalScore = self.alignMatrix[len(self.seq1)][len(self.seq2)]
        self.makeAlignment()
        
        
        
    def makeAlignment(self):
        
        s1 = ''
        s2 = ''
        
        i = len(self.seq1)
        j = len(self.seq2)
        alignType = self.traceBackMatrix[i][j]
        
        while alignType != 'f':
            
            if alignType == 'd':
                s1 = self.seq1[i-1] + s1
                s2 = self.seq2[j-1] + s2
                i -= 1
                j -= 1
                
            if alignType == 'l':
                s1 = '-' + s1
                s2 = self.seq2[j-1] + s2
                j -= 1
                
            if alignType == 'u':
                s1 = self.seq1[i-1] + s1
                s2 = '-' + s2
                i -= 1
                
            alignType = self.traceBackMatrix[i][j]
            
        self.s1 = s1
        self.s2 = s2
        
        self.getIdentity()
        
        
    def getIdentity(self):
        
        ident = 0
        
        for i in range(len(self.s1)):
            if self.s1[i] == self.s2[i]:
                ident += 1
        
        self.identity = ident
    
    
    def printResults(self):
        print(self.alignMatrix, '\n')
        print(self.s1)
        print(self.s2,'\n')
        print('Final Score:', self.finalScore)
        print('Identity:', self.identity)
        
        
        

In [119]:
aligner = Aligner("AATCG","AACG", -2, -1, 1)
aligner.align()
aligner.printResults()

[[  0  -2  -4  -6  -8]
 [ -2   1  -1  -3  -5]
 [ -4  -1   2   0  -2]
 [ -6  -3   0   1  -1]
 [ -8  -5  -2   1   0]
 [-10  -7  -4  -1   2]] 

AATCG
AA-CG 

Final Score: 2
Identity: 4
